In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time


In [ ]:
import random
import pandas as pd
random.seed(101)

df_test_dialect_msa = pd.DataFrame(columns=["ex_id",'Dialect_Original', "GPT4_MSA_pred",	"Gemini_MSA_pred",	"ClaudeOpus_MSA_pred",	"Llama70B_MSA_pred"])

# add data to Dialect_Original


# Claude Opus

In [ ]:
%pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="[API_KEY]"
)

In [ ]:
source_language = "Coptic"
target_language = "Arabic"
counter = 0

In [ ]:
for index, raw in tqdm(df_test_dialect_msa[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  zero_shot_example = f"""### Translate this sentence from {source_language} to {target_language} (generate only the {target_language} translation), Source:
  {raw[f"{source_language}_Original"]}
  ### Target:
  """
  try:
    message = client.messages.create(
      model="claude-3-opus-20240229",
      max_tokens=1000,
      temperature=0,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": zero_shot_example
                  }
              ]
          }
      ]
    )
    df_test_dialect_msa.loc[index, f"ClaudeOpus_{target_language}_pred"] = message.content[0].text
    counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 30 seconds")
    time.sleep(30)
    message = client.messages.create(
      model="claude-3-opus-20240229",
      max_tokens=1000,
      temperature=0,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": zero_shot_example
                  }
              ]
          }
      ]
    )
    df_test_dialect_msa.loc[index, f"ClaudeOpus_{target_language}_pred"] = message.content[0].text
    counter += 1


101it [07:25,  4.41s/it]


In [ ]:
source_language = "Coptic"
target_language = "Arabic"
counter = 0

for index, raw in tqdm(test100_5shots_arabic[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  selected = test100_5shots_arabic.iloc[int(index)]

  source_examples = []
  target_examples = []

  for i in range(5):
    source_examples.append(selected.loc[f'{source_language}_shot_{i}'].replace("\n",""))
    target_examples.append(selected.loc[f'{target_language}_shot_{i}'].replace("\n",""))

  source_sentence = selected.loc[f'{source_language}_Original'].replace("\n","")
  target_sentence = selected.loc[f'{target_language}_Original'].replace("\n","")


  five_shot_example = f"""Translate this from {source_language} into 1. {target_language} (generate only the {target_language} for number 6):
  {source_examples[0]}
  1. {target_examples[0]}
  Translate this from {source_language} into 2. {target_language}:
  {source_examples[1]}
  2. {target_examples[1]}
  Translate this from {source_language} into 3. {target_language}:
  {source_examples[2]}
  3. {target_examples[2]}
  Translate this from {source_language} into 4. {target_language}:
  {source_examples[3]}
  4. {target_examples[3]}
  Translate this from {source_language} into 5. {target_language}:
  {source_examples[4]}
  5. {target_examples[4]}
  Translate this from {source_language} into 6. {target_language}:
  {source_sentence}
  6.
  """

  try:
    message = client.messages.create(
      model="claude-3-opus-20240229",
      max_tokens=1000,
      temperature=0,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": five_shot_example
                  }
              ]
          }
      ]
    )
    # print(message.content[0].text.replace("6.", ""))
    # break
    test100_5shots_arabic.loc[index, f"ClaudeOpus_{target_language}_pred"] = message.content[0].text.replace("6.", "")
    counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 30 seconds")
    time.sleep(30)
    message = client.messages.create(
      model="claude-3-opus-20240229",
      max_tokens=1000,
      temperature=0,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": five_shot_example
                  }
              ]
          }
      ]
    )
    test100_5shots_arabic.loc[index, f"ClaudeOpus_{target_language}_pred"] = message.content[0].text.replace("6.", "")
    counter += 1


  # print(zero_shot_example)
  # print(message.content[0].text)

#gemini pro

In [ ]:
%pip3 install --upgrade --user --quiet google-cloud-aiplatform

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

In [ ]:
MODEL_ID = "gemini-1.5-pro-preview-0514"  # @param {type:"string"}

model = GenerativeModel(MODEL_ID)

In [ ]:
# Load a example model with system instructions
example_model = GenerativeModel(
    MODEL_ID,
    # system_instruction=[
    #     "You are a helpful language translator.",
    #     "Your mission is to translate text in English to French.",
    # ],
)

# Set model parameters
generation_config = GenerationConfig(
    temperature=0.5,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

# Set safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

# safe = [
#     {
#         "category": "HARM_CATEGORY_HARASSMENT",
#         "threshold": "BLOCK_NONE",
#     },
#     {
#         "category": "HARM_CATEGORY_HATE_SPEECH",
#         "threshold": "BLOCK_NONE",
#     },
#     {
#         "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
#         "threshold": "BLOCK_NONE",
#     },
#     {
#         "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
#         "threshold": "BLOCK_NONE",
#     },
# ]

prompt = """
  User input: I like bagels.
  Answer:
"""

# Set contents to send to the model
contents = [prompt]

# Counts tokens
print(example_model.count_tokens(contents))

# Prompt the model to generate content
response = example_model.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

# Print the model response
print(f"\nAnswer:\n{response.text}")

total_tokens: 14
total_billable_characters: 29


Answer:
It's great that you like bagels! They are delicious and versatile. 

Is there anything else you'd like to tell me about your love for bagels? For example:

* What kind of bagels are your favorite? 
* What do you like to put on your bagels? 
* Where do you get your bagels from? 

Let's talk bagels! 😊 



In [ ]:
source_language = "Dialect"
target_language = "MSA"
counter = 0
finished_counter = 0

In [ ]:
for index, raw in tqdm(df_test_dialect_msa[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  zero_shot_example = f"""### Translate this sentence from {source_language} to {target_language} (generate only the {target_language} translation), Source:
  {raw[f"{source_language}_Original"]}
  ### Target:
  """
  try:
    contents = [zero_shot_example]
    response = example_model.generate_content(
        contents,
        generation_config=generation_config,
        # safety_settings=safety_settings,
    )
    if str(response.candidates[0].finish_reason) == "FinishReason.STOP":
      df_test_dialect_msa.loc[index, f"Gemini_{target_language}_pred"] = response.text
      counter += 1
      finished_counter += 1
    else:
      df_test_dialect_msa.loc[index, f"Gemini_{target_language}_pred"] = "No Answer"
      counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 50 seconds")
    time.sleep(100)
    contents = [zero_shot_example]
    response = example_model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    # print(response.text)
    df_test_dialect_msa.loc[index, f"Gemini_{target_language}_pred"] = response.text
    counter += 1
    # response = client.chat.completions.create(
    #   model="gpt-4-turbo-2024-04-09",
    #   messages=[
    #     {
    #       "role": "user",
    #       "content": ""
    #     }
    #   ],
    #   temperature=1,
    #   max_tokens=256,
    #   top_p=1,
    #   frequency_penalty=0,
    #   presence_penalty=0
    # )
    # dev100_english.loc[index, f"Gemini_{target_language}_pred"] = response.choices[0].message.content
    # counter += 1


  # print(zero_shot_example)
  # print(message.content[0].text)

In [ ]:
source_language = "Arabic"
target_language = "English"
counter = 0

for index, raw in tqdm(test100_5shots_arabic_english[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  selected = test100_5shots_arabic_english.iloc[int(index)]

  source_examples = []
  target_examples = []

  for i in range(5):
    source_examples.append(selected.loc[f'{source_language}_shot_{i}'].replace("\n",""))
    target_examples.append(selected.loc[f'{target_language}_shot_{i}'].replace("\n",""))

  source_sentence = selected.loc[f'{source_language}_Original'].replace("\n","")
  target_sentence = selected.loc[f'{target_language}_Original'].replace("\n","")


  five_shot_example = f"""Translate this from {source_language} into 1. {target_language} (generate only {target_language} translation for number 6):
  {source_examples[0]}
  1. {target_examples[0]}
  Translate this from {source_language} into 2. {target_language}:
  {source_examples[1]}
  2. {target_examples[1]}
  Translate this from {source_language} into 3. {target_language}:
  {source_examples[2]}
  3. {target_examples[2]}
  Translate this from {source_language} into 4. {target_language}:
  {source_examples[3]}
  4. {target_examples[3]}
  Translate this from {source_language} into 5. {target_language}:
  {source_examples[4]}
  5. {target_examples[4]}
  Translate this from {source_language} into 6. {target_language}:
  {source_sentence}
  6.
  """
  try:
    contents = [five_shot_example]
    response = example_model.generate_content(
        contents,
        generation_config=generation_config,
        # safety_settings=safety_settings,
    )
    if str(response.candidates[0].finish_reason) == "FinishReason.STOP":
      test100_5shots_arabic_english.loc[index, f"Gemini_{target_language}_pred"] = response.text
      counter += 1
      finished_counter += 1
    else:
      test100_5shots_arabic_english.loc[index, f"Gemini_{target_language}_pred"] = "No Answer"
      counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 50 seconds")
    time.sleep(100)
    while True:
      try:
              contents = [five_shot_example]
              response = example_model.generate_content(
                  contents,
                  generation_config=generation_config,
                  safety_settings=safety_settings,
              )
              # print(response.text)
              test100_5shots_arabic_english.loc[index, f"Gemini_{target_language}_pred"] = response.text
              counter += 1
      except ValueError:
          print(f"overloading error at {counter}, sleeping for 50 seconds")
      else:
          break

  # print(zero_shot_example)
  # print(message.content[0].text)

# GPT4

In [ ]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
client = OpenAI(api_key= "")

In [ ]:
source_language = "Dialect"
target_language = "MSA"
counter = 0

In [ ]:
for index, raw in tqdm(df_test_dialect_msa[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  zero_shot_example = f"""### Translate this sentence from {source_language} to {target_language} (generate only the {target_language} translation), Source:
  {raw[f"{source_language}_Original"]}
  ### Target:
  """
  try:
    response = client.chat.completions.create(
      model="gpt-4-turbo-2024-04-09",
      messages=[
        {
          "role": "user",
          "content": zero_shot_example
        }
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
  # print(response.choices[0].message.content)
    df_test_dialect_msa.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content
    counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 30 seconds")
    time.sleep(30)
    response = client.chat.completions.create(
      model="gpt-4-turbo-2024-04-09",
      messages=[
        {
          "role": "user",
          "content": zero_shot_example
        }
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
  # print(response.choices[0].message.content)
    df_test_dialect_msa.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content
    counter += 1

  # print(zero_shot_example)
  # print(message.content[0].text)

101it [05:37,  3.34s/it]


In [ ]:
source_language = "Coptic"
target_language = "Arabic"
counter = 0

for index, raw in tqdm(test100_5shots_arabic[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  selected = test100_5shots_arabic.iloc[int(index)]

  source_examples = []
  target_examples = []

  for i in range(5):
    source_examples.append(selected.loc[f'{source_language}_shot_{i}'].replace("\n",""))
    target_examples.append(selected.loc[f'{target_language}_shot_{i}'].replace("\n",""))

  source_sentence = selected.loc[f'{source_language}_Original'].replace("\n","")
  target_sentence = selected.loc[f'{target_language}_Original'].replace("\n","")


  five_shot_example = f"""Translate this from {source_language} into 1. {target_language} (generate only the {target_language} for number 6):
  {source_examples[0]}
  1. {target_examples[0]}
  Translate this from {source_language} into 2. {target_language}:
  {source_examples[1]}
  2. {target_examples[1]}
  Translate this from {source_language} into 3. {target_language}:
  {source_examples[2]}
  3. {target_examples[2]}
  Translate this from {source_language} into 4. {target_language}:
  {source_examples[3]}
  4. {target_examples[3]}
  Translate this from {source_language} into 5. {target_language}:
  {source_examples[4]}
  5. {target_examples[4]}
  Translate this from {source_language} into 6. {target_language}:
  {source_sentence}
  6.
  """

  try:
    response = client.chat.completions.create(
      model="gpt-4-turbo-2024-04-09",
      messages=[
        {
          "role": "user",
          "content": five_shot_example
        }
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )

    # print(response.choices[0].message.content)
    # break
    # dev100_english.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content
    test100_5shots_arabic.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content.replace("6.", "")
    counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 30 seconds")
    time.sleep(30)
    response = client.chat.completions.create(
      model="gpt-4-turbo-2024-04-09",
      messages=[
        {
          "role": "user",
          "content": five_shot_example
        }
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )

    # print(response.choices[0].message.content)
    # break
    # dev100_english.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content
    test100_5shots_arabic.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content.replace("6.", "")
    counter += 1


  # print(zero_shot_example)
  # print(message.content[0].text)

# llama3

In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq

client = Groq(
    api_key= "",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's AI landscape, and their importance can be summarized as follows:

1. **Real-time Processing**: Fast language models enable real-time processing of natural language inputs, which is essential for applications that require immediate responses, such as:
	* Conversational AI, chatbots, and virtual assistants (e.g., Siri, Alexa, Google Assistant)
	* Sentiment analysis and opinion mining in social media monitoring
	* Language translation and interpretation in multilingual settings
2. **Low Latency**: Fast language models minimize latency, which is critical in applications where timely responses are vital, such as:
	* Real-time sentiment analysis for stock market predictions
	* Language-based game development (e.g., language-based puzzles, word games)
	* Real-time language translation in video conferencing or online meetings
3. **Scalability**: Fast language models enable the processing of large volumes of text data, making them suitable for applica

In [ ]:
source_language = "Dialect"
target_language = "MSA"
counter = 0

In [ ]:
for index, raw in tqdm(df_test_dialect_msa[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  zero_shot_example = f"""### Translate this sentence from {source_language} to {target_language} (generate only the {target_language} translation), Source:
  {raw[f"{source_language}_Original"]}
  ### Target:
  """
  try:
    chat_completion = client.chat.completions.create(
      messages=[
              {
                  "role": "user",
                  "content": zero_shot_example,
              }
          ],
          model="llama3-70b-8192",
      )
  # print(chat_completion.choices[0].message.content)
    df_test_dialect_msa.loc[index, f"Llama70B_{target_language}_pred"] = chat_completion.choices[0].message.content
    counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 30 seconds")
    time.sleep(30)
    chat_completion = client.chat.completions.create(
      messages=[
              {
                  "role": "user",
                  "content": zero_shot_example,
              }
          ],
          model="llama3-70b-8192",
      )
  # print(chat_completion.choices[0].message.content)
    df_test_dialect_msa.loc[index, f"Llama70B_{target_language}_pred"] = chat_completion.choices[0].message.content
    counter += 1

  # print(zero_shot_example)
  # print(message.content[0].text)

101it [03:21,  2.00s/it]


In [ ]:
source_language = "Coptic"
target_language = "Arabic"
counter = 0

for index, raw in tqdm(test100_5shots_arabic[counter:].iterrows()):
  # print(raw['English_Original'])
  # print(raw['Coptic_Original'])
  selected = test100_5shots_arabic.iloc[int(index)]

  source_examples = []
  target_examples = []

  for i in range(5):
    source_examples.append(selected.loc[f'{source_language}_shot_{i}'].replace("\n",""))
    target_examples.append(selected.loc[f'{target_language}_shot_{i}'].replace("\n",""))

  source_sentence = selected.loc[f'{source_language}_Original'].replace("\n","")
  target_sentence = selected.loc[f'{target_language}_Original'].replace("\n","")


  five_shot_example = f"""Translate this from {source_language} into 1. {target_language} (generate only the {target_language} for number 6):
  {source_examples[0]}
  1. {target_examples[0]}
  Translate this from {source_language} into 2. {target_language}:
  {source_examples[1]}
  2. {target_examples[1]}
  Translate this from {source_language} into 3. {target_language}:
  {source_examples[2]}
  3. {target_examples[2]}
  Translate this from {source_language} into 4. {target_language}:
  {source_examples[3]}
  4. {target_examples[3]}
  Translate this from {source_language} into 5. {target_language}:
  {source_examples[4]}
  5. {target_examples[4]}
  Translate this from {source_language} into 6. {target_language}:
  {source_sentence}
  6.
  """

  try:
    chat_completion = client.chat.completions.create(
        messages=[
                {
                    "role": "user",
                    "content": five_shot_example,
                }
            ],
            model="llama3-70b-8192",
        )

    # print(response.choices[0].message.content)
    # break
    # dev100_english.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content
    test100_5shots_arabic.loc[index, f"Llama70B_{target_language}_pred"] = chat_completion.choices[0].message.content.replace("6.", "")
    counter += 1
  except:
    print(f"overloading error at {counter}, sleeping for 30 seconds")
    time.sleep(30)
    chat_completion = client.chat.completions.create(
        messages=[
                {
                    "role": "user",
                    "content": five_shot_example,
                }
            ],
            model="llama3-70b-8192",
        )

    # print(response.choices[0].message.content)
    # break
    # dev100_english.loc[index, f"GPT4_{target_language}_pred"] = response.choices[0].message.content
    test100_5shots_arabic.loc[index, f"Llama70B_{target_language}_pred"] = chat_completion.choices[0].message.content.replace("6.", "")
    counter += 1


  # print(zero_shot_example)
  # print(message.content[0].text)